### Navigation
1. [Day 11](#Day-11)  
1. [Day 12](#Day-12)  
1. [Day 13](#Day-13)  

# Day 11
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/11

### --- Part One ---

In [1]:
import numpy as np

with open("data/day11-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]
    inputs = np.asarray([[int(c) for c in r] for r in inputs])
    inputs = np.pad(inputs, [(1,1),(1,1)], constant_values=11)

In [2]:
inputs

array([[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
       [11,  1,  2,  2,  4,  3,  4,  6,  3,  8,  4, 11],
       [11,  5,  6,  2,  1,  1,  2,  8,  5,  8,  7, 11],
       [11,  6,  3,  8,  8,  4,  2,  6,  5,  4,  6, 11],
       [11,  1,  5,  5,  6,  2,  4,  7,  7,  5,  6, 11],
       [11,  1,  4,  5,  1,  8,  1,  1,  5,  7,  3, 11],
       [11,  1,  8,  3,  2,  3,  8,  8,  1,  2,  2, 11],
       [11,  2,  7,  4,  8,  5,  4,  5,  6,  4,  7, 11],
       [11,  2,  5,  8,  2,  8,  7,  7,  4,  3,  2, 11],
       [11,  3,  1,  8,  5,  6,  4,  3,  8,  7,  1, 11],
       [11,  2,  2,  2,  4,  8,  7,  6,  6,  2,  7, 11],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]])

In [3]:
def recursion(inputs, zero_points=[]):
    # get all with power 10
    power10 = np.where(inputs==10)
    power10_pts = list(zip(power10[0], power10[1]))
    # move all the 10s to 11s so that we dont count them anymore
    for pt in power10_pts:
        inputs[pt] += 1
    zero_points += power10_pts
    for r, c in power10_pts:
        inputs[r-1:r+2, c-1:c+2] += 1
        inputs, zero_points = recursion(inputs, zero_points)
    return inputs, zero_points

In [4]:
flash_counter = 0
i = 0
while(True):
    i += 1
    # increase power by 1
    inputs += 1
    inputs, zero_points = recursion(inputs, zero_points=[])
    # increase flash counter
    flash_counter += len(zero_points)
    # replace flashed points with zeros
    for pt in zero_points:
        inputs[pt] = 0
    if i == 100:
        print("Answer for part one:", flash_counter)
    if len(zero_points) == 100:
        print("Answer for part two:", i)
        break
    

Answer for part one: 1591
Answer for part two: 314


# Day 12
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/12

### --- Part One ---

In [5]:
with open("data/day12-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [6]:
from collections import Counter

def add_to_graph(graph, a, b):
    if a in graph.keys():
        graph[a] += [b]
    else:
        graph[a] = [b]
    return graph

def check_revisit_possible(path):
    path = [cave for cave in path
            if str.islower(cave) and cave not in ['start', 'end']]
    counts = Counter(path)
    if sum(counts.values()) == len(counts.keys()):
        return True
    return False

def count_paths(curr_paths, graph, paths_counter, part_two=True):
    for curr_path in curr_paths:
        last_position = curr_path[-1]
        posible_moves = graph[last_position]
        if part_two:
            revisit = check_revisit_possible(curr_path)
        for pos_move in posible_moves:
            if pos_move == 'end':
                paths_counter += 1
            elif (str.isupper(pos_move) or pos_move not in curr_path
                  or (part_two
                      and pos_move != 'start'
                      and revisit)):
                # recursion
                paths_counter = count_paths(
                    [curr_path + [pos_move]], graph, paths_counter, part_two)
    return paths_counter

In [7]:
graph = {}
for mapping in inputs:
    a, b = mapping.split('-')
    add_to_graph(graph, a, b)
    add_to_graph(graph, b, a)

In [8]:
%%time
all_paths = count_paths([['start']], graph, 0, False)
all_paths

Wall time: 17 ms


3802

### --- Part Two ---

In [9]:
%%time
all_paths = count_paths([['start']], graph, 0, True)
all_paths

Wall time: 1.29 s


99448

# Day 13
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/13

### --- Part One ---

In [10]:
with open("data/day13-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines() if line != '\n']
    dots = [[int(v) for v in line.split(',')] for line in inputs if ',' in line]
    instructions = [line.split('=') for line in inputs if '=' in line]

In [11]:
import numpy as np

dots_np = np.asarray(dots)

x_max = np.asarray(dots_np)[:,0].max() + 1
y_max = np.asarray(dots_np)[:,1].max() + 1

x_max = x_max + 1 if x_max % 2 == 0 else x_max
y_max = y_max + 1 if y_max % 2 == 0 else y_max

dots_paper_np = np.zeros((y_max, x_max))
for x, y in dots_np:
    dots_paper_np[y, x] = 1

In [12]:
def fold_paper(paper, instr, v):
    if 'x' in instr:
        paper_a = paper[:,:v]
        paper_b = np.fliplr(paper[:,v+1:])
    elif 'y' in instr:
        paper_a = paper[:v,:]
        paper_b = np.flipud(paper[v+1:,:])
    return np.logical_or(paper_a, paper_b)

In [13]:
folded_paper = dots_paper_np.copy()
for i, instr in enumerate(instructions[:]):
    instr, v = instr
    folded_paper = fold_paper(folded_paper, instr, int(v))
    if i == 0:
        print("Answer part one:",folded_paper.sum())

Answer part one: 737


### --- Part Two ---

In [14]:
for i, v in enumerate(range(4,40,5)):
    print("---Letter ", i+1, "---")
    print(folded_paper[:,v-4:v] * 1)

---Letter  1 ---
[[1 1 1 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 1 1 1]]
---Letter  2 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  3 ---
[[0 0 1 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  4 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  5 ---
[[0 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  6 ---
[[1 1 1 1]
 [1 0 0 0]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]
---Letter  7 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  8 ---
[[1 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]]
